<h3> Model Registry Initiation </h3>

In [1]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import mlflow
import os


<h3> Set mlflow tracking uri and experiment </h3>

In [2]:
os.environ["AWS_PROFILE"] = "capstone-profile" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "ec2-3-16-146-5.us-east-2.compute.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [7]:
mlflow.search_experiments()

MlflowException: API request to http://ec2-3-16-146-5.us-east-2.compute.amazonaws.com:5000/api/2.0/mlflow/experiments/search failed with exception HTTPConnectionPool(host='ec2-3-16-146-5.us-east-2.compute.amazonaws.com', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/search (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1297c3eb0>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [6]:
import pandas as pd
import mlflow
import pickle

<h3> Read Pickled dataframees </h3>

In [7]:
df_full = pd.read_pickle("./df_full.pkl")
df_train = pd.read_pickle("./df_train.pkl")
df_val = pd.read_pickle("./df_val.pkl")
df_test = pd.read_pickle("./df_test.pkl")

y_train = pd.read_pickle("./y_train.pkl")
y_val = pd.read_pickle("./y_val.pkl")
y_test = pd.read_pickle("./y_test.pkl")


<h3> Dict Vectorizer </h3>

In [8]:
from sklearn.feature_extraction import DictVectorizer

In [9]:
numerical = ['registration_init_time', 'num_25', 'num_50', 'num_75', 'num_985', 'num_100', 'num_unq', 'total_secs']

In [10]:
categorical = ['gender', 'registered_via', 'city']

In [11]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
test_dicts = df_test[categorical + numerical].to_dict(orient='records')

In [12]:
dv = DictVectorizer(sparse=False)

In [13]:
X_train = dv.fit_transform(train_dicts)
X_val = dv.fit_transform(val_dicts)
X_test = dv.fit_transform(test_dicts)

<h2> Training Logistic Regression </h2>

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
mlflow.sklearn.autolog()

mlflow.set_experiment("kkbox-churn-prediction")

# run = mlflow.active_run()
# if(run.info.status == 'RUNNING'):
#    mlflow.end_run()

with mlflow.start_run():
    mlflow.set_tag("developer", "st")

    model = LogisticRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    score = roc_auc_score(y_val, y_pred)
# score

    y_pred = model.predict(X_test)
    score = roc_auc_score(y_test, y_pred)

    mlflow.log_metric("roc_auc_score", score)

mlflow.end_run()

MlflowException: API request to http://ec2-3-16-146-5.us-east-2.compute.amazonaws.com:5000/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='ec2-3-16-146-5.us-east-2.compute.amazonaws.com', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=kkbox-churn-prediction (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x29e836130>: Failed to establish a new connection: [Errno 60] Operation timed out'))